In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
from evaluations import *
from utils import *
data='traffic'
m2 = np.load('../data/%s/M2.npy'%data)

In [3]:
np.max(m2)

13547.346

## movsim data

In [1]:
from evaluations import *
from utils import *
data='geolife'
train_data = read_data_from_file('../data/%s/real.data'%data)
gps = get_gps('../data/%s/gps'%data)
m2 = np.load('../data/%s/M2.npy'%data)

In [42]:
m2.shape

(23768, 23768)

In [43]:
m2[0][0:10]

array([0.        , 0.006     , 0.02080865, 0.0256125 , 0.03023243,
       0.02830194, 0.02968164, 0.10960383, 0.12234786, 0.06280127])

In [10]:
import numpy as np
start_data = np.load('../data/geolife/start.npy')

In [11]:
np.sum(start_data)

1.0

In [29]:
import torch
torch.multinomial(torch.Tensor(start_data[:10]), 1) 

tensor([5])

## traffic data

In [9]:
import pandas as pd
df = pd.read_csv("../data/geolife/final.csv", index_col=0)
main_df = df
print(df.columns)
edge_cols = ['eid', 'lon1', 'lat1', 'lon2', 'lat2']
edge_df = df[edge_cols]
edge_df.drop_duplicates(inplace=True)
edge_df.reset_index(inplace=True)
edge_df.to_csv('../data/traffic/edges.csv')
select_cols = ['vehicle_id', 'speed', 'timestamp', 'eid']
df = df[select_cols]
print(df.head())

/home/xjm/anaconda3/envs/movesim/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Index(['vehicle_id', 'speed', 'timestamp', 'lon', 'lat', 'hash_x', 'pid',
       'eid', 'dist', 'lon1', 'lat1', 'lon2', 'lat2', 'hash_y', 'hash1',
       'hash2'],
      dtype='object')


/home/xjm/anaconda3/envs/movesim/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


   vehicle_id  speed  timestamp    eid
0       81044      0          0  78910
1       81044      0          1  78910
2       81044      0          2  78910
3       81044      0          3  78910
4       81044      0          4  78910


In [11]:
len(df['eid'].unique()) # 边的数量

46165

In [20]:
def roll_mean(x:pd.DataFrame):
    x['speed'] = x['speed'].rolling(window=3, center=True).mean()
    x['speed'] = x['speed'].fillna(0)
    return x

In [21]:
roll_df = df.groupby('vehicle_id').apply(roll_mean)

In [43]:
roll_df.head()

,vehicle_id,speed,timestamp,eid
0,81044,NaN,0,78910
1,81044,0.0,1,78910
2,81044,0.0,2,78910
3,81044,0.0,3,78910
4,81044,0.0,4,78910


In [58]:
grouped = roll_df.groupby('vehicle_id')

In [33]:
import pandas as pd
import numpy as np
import json
import tqdm
def gen_traj(x:pd.DataFrame, seq_len=48, interval_len=4, alpha=0.4):
    """从速度不为0的点开始生成

    Args:
        x (pd.DataFrame): _description_
        seq_len (int, optional): 序列长度. Defaults to 48.
        interval_len (int, optional): 生成序列的间隔长度. Defaults to 4.
        alpha (float, optional): 满足不为0的数量. Defaults to 0.4.
    """ 
    temp_df = x[x['speed']!=0]
    #print(len(temp_df))
    if len(temp_df) == 0:
        return None
    start_pos = list(temp_df['timestamp'])[0]  
    ans = []
    for i in range(start_pos, len(x)-seq_len, interval_len):
        seq = x.iloc[i: i+seq_len]
        if (len(seq[seq['speed']!=0]))/seq_len > alpha:
            ans.append(list(seq['eid']).copy())
    if len(ans) == 0:
        return None
    return np.array(ans)

      
def preprocess(file):
    df = pd.read_csv(file, index_col=0)
    select_cols = ['vehicle_id', 'speed', 'timestamp', 'eid']
    df = df[select_cols]
    #df = df[df['speed']!=0]
    print("eid count: ", len(df['eid'].unique()))
    print("total count: ", len(df['eid']))
    # exclude long-stop point
    # mean roll for traj
    def roll_mean(x:pd.DataFrame):
        x['speed'] = x['speed'].rolling(window=3, center=True).mean()
        x['speed'] = x['speed'].fillna(0)
        return x
    roll_df = df.groupby('vehicle_id').apply(roll_mean)
    # generate trajectory of each vehicle
    grouped = roll_df.groupby('vehicle_id')
    vehicle2traj = dict()
    for key, group in tqdm.tqdm(grouped):
        traj = gen_traj(group)
        if traj is not None:
            vehicle2traj[key] = traj
        else:
            pass
    # save data
    veh2ind = {}
    trajs = []
    start_ind = 0
    for key in vehicle2traj.keys():
        veh2ind[str(key)] = {'start_ind':start_ind, 'len':len(vehicle2traj[key])}
        start_ind = start_ind + len(vehicle2traj[key])
        trajs.append(vehicle2traj[key])
    trajs = np.vstack(trajs)
    np.save('../data/trajs.npy', trajs)
    with open('../data/veh2ind', 'w') as f:
        json.dump(veh2ind, f)
    print("successfully preprocess data")
    

In [34]:
preprocess("../data/geolife/final.csv")

/home/xjm/anaconda3/envs/movesim/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


eid count:  46165
total count:  6215040



100%|██████████| 4316/4316 [11:34<00:00,  6.63it/s]


successfully preprocess data


## load data

In [4]:
def read_file(data_file):
    with open(data_file, 'r') as f:
        lines = f.readlines()
    lis = []
    for line in lines:
        l = line.strip().split(' ')
        l = [int(s) for s in l]
        lis.append(l)
    return lis
data = read_file(r'../data/geolife/real.data')
fake_data = read_file(r'../data/geolife/dispre.data')

In [12]:
from traffic import gen_eid_dict
gen_eid_dict()

In [2]:
import json
with open('../data/traffic/eid2id.json', 'r') as f:
    eid2id = json.load(f) 
len(eid2id)

46165

In [3]:
import numpy as np
m = np.zeros(46165, dtype=np.int32)
np.save('M2.npy', m)

In [53]:
import pandas as pd
edge_df = pd.read_csv('../data/traffic/edges.csv', index_col=0)

In [58]:
edge_df.index[np.where(np.isnan(edge_df))[0]]

Int64Index([6499, 6499, 6499, 6499], dtype='int64')

In [60]:
edge_df.iloc[6499]

index    54063.0
eid         -1.0
lon1         NaN
lat1         NaN
lon2         NaN
lat2         NaN
Name: 6499, dtype: float64

In [45]:
m1 = edge_df[['lon1', 'lat1']].values
m2 = edge_df[['lon2', 'lat2']].values
print(m1.shape, m2.shape)

(46165, 2) (46165, 2)


In [12]:
m2 = np.load("../data/traffic/M2.npy")

In [13]:
np.max(m2)

1.2271162733629495

In [48]:
import torch.nn as nn 
import torch 
import numpy as np
import torch.nn.functional as F
mat = np.load("mat2.npy")
#mat = mat/mat.sum(axis=0)
print(np.max(mat))
device = torch.device("cuda:"+"2")
mat = torch.Tensor(mat)
mat = F.normalize(mat, dim=0)
# mat = torch.randn((32, 46165))
l1 = nn.Linear(46165, 64*2)
x = l1(mat) # 输出nan


nan


In [16]:
w = l1.weight
b = l1.bias

In [21]:
w.shape, b.shape, mat.shape

(torch.Size([128, 46165]), torch.Size([128]), torch.Size([32, 46165]))

In [9]:
import numpy as np 
s = np.array([[1, 2], [3, 4]])
# s = s.reshape(-1)
print(s)
def f(x):
    dic = {1:2, 2:3, 3:4, 4:1}
    return dic[x]
applyall = np.vectorize(f)
applyall(s)


[[1 2]
 [3 4]]


array([[2, 3],
       [4, 1]])

In [1]:
from traffic import gen_node_gps, gen_val_data

In [2]:
gen_val_data()

In [1]:
import numpy as np
x = np.load('x.npy')

In [3]:
x

array([[-10.810335 , -10.605946 , -10.846008 , ..., -10.645202 ,
        -10.852803 , -10.656425 ],
       [-10.8186455, -10.611062 , -10.843485 , ..., -10.6459875,
        -10.860435 , -10.656467 ],
       [-10.798337 , -10.606144 , -10.838633 , ..., -10.645353 ,
        -10.851723 , -10.658152 ],
       ...,
       [-10.80941  , -10.604166 , -10.847086 , ..., -10.635123 ,
        -10.853417 , -10.6645155],
       [-10.80995  , -10.607412 , -10.842242 , ..., -10.637983 ,
        -10.852778 , -10.660442 ],
       [-10.797813 , -10.608531 , -10.830216 , ..., -10.63449  ,
        -10.864302 , -10.653595 ]], dtype=float32)

In [5]:
x.shape

(32, 46165)